In [1]:
import random
from time import sleep
import gym
from gym import logger
import numpy as np
from utils import get_level
from PacmanAgent import PacmanAgent
from collections import defaultdict



In [2]:
env = PacmanAgent()


/home/hazhir-wsl/anaconda3/envs/PacmanOpenAIGym_FHDortmund/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (7, 7)
  logger.warn(


Box(0, 5, (7, 7), int32)

In [3]:
n_states = np.prod(env.reset().shape)
n_actions = env.action_space.n

print(n_states)
print(n_actions)

49
4


/home/hazhir-wsl/anaconda3/envs/PacmanOpenAIGym_FHDortmund/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/hazhir-wsl/anaconda3/envs/PacmanOpenAIGym_FHDortmund/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/hazhir-wsl/anaconda3/envs/PacmanOpenAIGym_FHDortmund/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(


In [29]:
q_table = np.zeros((n_states, n_actions))

In [52]:
np.max(q_table[env.reset()])

2.0

In [5]:
# Hyperparameters
alpha = 0.9          # Learning rate
gamma = 0.95         # Discount factor
epsilon = 1.0        # Exploration rate (epsilon-greedy)
epsilon_decay = 0.9995
epsilon_min = 0.01
num_episodes = 1000  # Total training episodes
max_steps = 100       # Max steps per episode

In [6]:
q_table = np.zeros((n_states, n_actions))

def flatten_position(env, state):
    # Find the agent's position in the grid
    for e in env.entities:
            if e.type == "pacman":
                position = np.array([e.x, e.y])
                break
    if position.size == 0:
        raise ValueError("Agent position not found in the grid!")
    row, col = position  # Extract the first (and only) match
    n_cols = state.shape[1]
    return row * n_cols + col  # Convert (row, col) to a single index

def extract_indices(flattened_index, n_rows, n_cols, n_actions):
    i = flattened_index // (n_cols * n_actions)  # Row index
    remainder = flattened_index % (n_cols * n_actions)
    j = remainder // n_actions  # Column index
    k = remainder % n_actions  # Action index
    return k

def choose_action(state):
    if random.uniform(0, 1) < epsilon:
        # Explore: choose a random action
        return env.action_space.sample()
    else:
        action = np.argmax(q_table[state, :])
        return extract_indices(action, 7, 7, 4)


def update_q_table(state, action, reward, next_state):
    best_next_action = np.argmax(q_table[next_state, :])
    try:
        td_target = reward + gamma * q_table[next_state, best_next_action]
    except Exception as e:
        print(q_table)
        print(next_state)
        raise e
    td_error = td_target - q_table[state, action]
    q_table[state, action] += alpha * td_error

for episode in range(num_episodes):
    state = env.reset()
    done = False
    
    for step in range(max_steps):
        action = choose_action(state)
        next_state, reward, done, info = env.step(action)
        next_state = flatten_position(env, next_state)
        update_q_table(state, action, reward, next_state)
        state = next_state
        
        if done:
            break
    
    # Decay epsilon
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

/home/hazhir-wsl/anaconda3/envs/PacmanOpenAIGym_FHDortmund/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/hazhir-wsl/anaconda3/envs/PacmanOpenAIGym_FHDortmund/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:225: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


In [9]:

# Play the game using the learned policy
env = PacmanAgent()
max_steps = 100
for episode in range(5):
    state = env.reset()
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        
        action = np.argmax(q_table[state, :])
        action = extract_indices(action, 7, 7, 4)
        next_state, reward, done, info = env.step(action)
        env.render(action)
        total_rewards += reward
        state = next_state
        sleep(0.3)
        if done:
            print(f"Episode: {episode + 1}, Total Reward: {total_rewards}")
            break

env.close()

Episode: 1, Total Reward: 0.5
Episode: 2, Total Reward: 0.5
Episode: 3, Total Reward: 0.5
Episode: 4, Total Reward: 0.5
Episode: 5, Total Reward: 0.5
